In [644]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import tensorflow as tf
import pandas as pd
import joblib as jb
from Evaluator import ModelEvaluator

In [645]:
 model=jb.load('rfDefense2021.sav')
 print(model)

RandomForestClassifier(max_depth=50, n_estimators=250, random_state=45)


In [646]:
data=pd.read_csv('datasets/malicious_data_generated.csv')
npdata=data.to_numpy()
malData=np.copy(npdata)
print(type(malData[1,:]))
print(malData[:1].shape)

<class 'numpy.ndarray'>
(1, 10)


In [647]:
class MalwareEnv():
    def __init__(self):
        # Actions we can take, decrease, increse, none
        self.action_space = Discrete(10)
        # max-min array
        self.observation_space = Box(low=-100,high=100,shape=(10,))
        
        # Set start 
        self.state = np.asarray(malData[random.randint(0,499),:])
        
        # Set time 
        self.length = 10
        
        self.evaluator=ModelEvaluator()
        
    def step(self, action):
        # Apply action for each state
        
        if(action<10):
           if(action==0):
               self.state[0]=round(random.uniform(-0.290698,133.441860),6)
           elif(action==1):
                self.state[1]=round(random.uniform(0,1184),6)
           elif(action==2):
                self.state[2]=round(random.uniform(-0.666667,10.666667),6)
           elif(action==3):
                self.state[3]=round(random.uniform(-0.312383,109.259173),6)
           elif(action==4):
                self.state[4]=round(random.uniform(0,30),6)
           elif(action==5):
                self.state[5]=round(random.uniform(-0.322,127.488889),6)
           elif(action==6):
                self.state[6]=round(random.uniform(-0.282353,147.976471),6)
           elif(action==7):
                self.state[7]=round(random.uniform(-0.164688,715.616633),6)
           elif(action==8):
                self.state[8]=round(random.uniform(-0.324081,106.407677),6)
           else:
                self.state[9]=round(random.uniform(-0.750000,227.5),6)  
        
        self.length -= 1 
            
        
        # Calculate reward in ranges
        
        value=self.state.tolist()
        result=self.evaluator.test(value)
        result=np.array2string(result)
        
        if(result=="['benign']"):
                 reward=+1
                 
        else:
                 reward=-1
        
        
        
            
                 
        #if(rewardMulti==0):
        #    reward=-1
        #    
        #else:
        #    reward=rewardMulti
                 
        
        # Check if is done
        if self.length <= 0:
            
            done = True
        else:
            done = False
        
        
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        pass
    
    def reset(self):
        # Reset 
        self.state = malData[random.randint(0,498),:]
        # Reset time
        self.length = 60 
        return self.state

In [648]:
env.observation_space.sample()

array([ 93.18925 , -25.524029,  11.785298,  65.14376 ,  46.20102 ,
        47.23928 ,  91.41796 ,  99.98343 ,  38.091476,  30.877542],
      dtype=float32)

In [649]:
env = MalwareEnv()

In [650]:
env.observation_space.sample()

array([-8.7883408e+01, -7.2571770e+01,  4.0549145e+01,  1.6392708e+01,
       -9.3764687e+01, -5.7374207e+01,  9.4603039e-02,  8.5831673e+01,
       -1.9213978e+01, -9.6324219e+01], dtype=float32)

In [651]:
with tf.device('/gpu:0'):
    episodes = 2
    for episode in range(1, episodes+1):
        state = env.reset()
        done = False
        score = 0 
    
        while not done:
            #env.render()
            action = env.action_space.sample()
            n_state, reward, done, info = env.step(action)
            score+=reward
        print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:60
Episode:2 Score:60


In [652]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow

In [653]:
states = env.observation_space.shape
actions = env.action_space.n

In [654]:
actions

10

In [655]:
states

(10,)

In [656]:
def build_model(states, actions):
    with tf.device('/gpu:0'):
        model = tensorflow.keras.models.Sequential()   
        model.add(tensorflow.keras.layers.Dense(24, activation='relu', input_shape=(1,10)))
        model.add(tensorflow.keras.layers.Dense(24, activation='relu'))
        model.add(tensorflow.keras.layers.Dense(10, activation='linear'))
        model.add(Flatten())

        return model

In [657]:

#with tf.device('/gpu:0'):
model = build_model(states, actions)

In [658]:
model.summary()

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_93 (Dense)             (None, 1, 24)             264       
_________________________________________________________________
dense_94 (Dense)             (None, 1, 24)             600       
_________________________________________________________________
dense_95 (Dense)             (None, 1, 10)             250       
_________________________________________________________________
flatten_31 (Flatten)         (None, 10)                0         
Total params: 1,114
Trainable params: 1,114
Non-trainable params: 0
_________________________________________________________________


In [659]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory


In [660]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=100, target_model_update=1e-2)
    return dqn

In [661]:

    dqn = build_agent(model, actions)
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])
    dqn.fit(env, nb_steps=10000, visualize=False, verbose=1)

Training for 10000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 322s 32ms/step - reward: 0.7424
done, took 322.020 seconds
